# Scraping aci.aero

In [21]:
#=========================
#        SETTINGS
#=========================

startYear = 2007
endYear = 2017

#=========================
import requests
import urllib.request
import re
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd

#Initialize
pageNum = startYear
yearList = []
rankList = []
airportCodeList = []
cityNameList = []
passengerTotalList = []
changePercentList = []
count = 0 #For tracing progress

#Loop
while pageNum <= endYear:
    
    if pageNum == 2017:
        url = 'https://aci.aero/data-centre/annual-traffic-data/passengers/' + str(pageNum) + '-passenger-summary-annual-traffic-data/'
    else:
        url = 'https://aci.aero/data-centre/annual-traffic-data/passengers/' + str(pageNum) + '-final-summary/'
    

    #Loading the page
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    results = BeautifulSoup(page).find(class_="row-hover").find_all("tr")
    
    for result in results:
        tds = result.find_all("td")
        
        #Rank
        rank = tds[0].text.strip()
        
        #City Name and Airport Code
        position = tds[1].text.find("(")
        cityAndAirport = tds[1].text.strip()
        airportCode = cityAndAirport[position+1 : -1]
        cityName = cityAndAirport[0:position].strip()
        
        #Passenger
        passengerTotal = tds[2].text.replace(" ", "").replace(",", "").strip()

        #Change Percent
        changePercent = tds[-1].text.replace("%", "").strip()
        
        #Write into the lists
        yearList.append(pageNum)
        rankList.append(rank)
        airportCodeList.append(airportCode)
        cityNameList.append(cityName)
        passengerTotalList.append(passengerTotal)
        changePercentList.append(changePercent)
            
    #Next Year
    pageNum += 1
    
    #Tracing progress
    print("{:.0%}".format(count/(endYear-startYear)) + " finished.")
    count +=1
    
#Output CSV
total = dict(zip(['Year',
                  'Rank', 
                  'City Name', 
                  'Airport Code', 
                  'Total Passengers', 
                  '% Change'], 
                 [yearList, 
                  rankList, 
                  cityNameList, 
                  airportCodeList, 
                  passengerTotalList,
                  changePercentList]))
df = pd.DataFrame(total)
df.to_csv("TenYearPassengerSummary.csv", index=False)
print("Done!")

0% finished.
10% finished.
20% finished.
30% finished.
40% finished.
50% finished.
60% finished.
70% finished.
80% finished.
90% finished.
100% finished.
Done!
